In [0]:
%run ./crm/silver_crm_cust_info
%sql
%python
%md
from utils.helpers import clean_customers

In [0]:
%sql
-- Spark SQL
SELECT * FROM workspace.silver.crm_cust_info

In [0]:
df = spark.table("workspace.silver.crm_cust_info")
df.display()


In [0]:
# Spark SQL Via PySpark
df = spark.sql("""
    SELECT
        gender,
        COUNT(*) AS total_customers
    FROM workspace.silver.crm_cust_info
    GROUP BY gender
""")

df.display()


In [0]:
from pyspark.sql import functions as F

df = spark.table("workspace.silver.crm_cust_info")

result_df = (
    df.groupBy("gender")
      .agg(F.count("*").alias("total_customers"))
)

result_df.display()


In [0]:
%sql
SELECT * FROM workspace.bronze.crm_cust_info VERSION AS OF 1

DESCRIBE HISTORY silver_customers

In [0]:
%sql
DESCRIBE HISTORY workspace.bronze.crm_cust_info

In [0]:
%sql
SELECT * FROM workspace.bronze.crm_cust_info VERSION AS OF 0

In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("silver.crm_cust_info")


df.write \
  .mode("append") \
  .format("delta") \
  .saveAsTable("silver.crm_cust_info")


from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "silver.crm_cust_info")

delta_table.alias("t") \
  .merge(
      df.alias("s"),
      "t.customer_id = s.customer_id"
  ) \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()
